In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam
from transformers import AutoTokenizer,AutoModel
from torch.utils.data import DataLoader,Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


c:\Users\danie\OneDrive\Desktop\Data_science\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
data_df = pd.read_json('Sarcasm_Headlines_Dataset.json',lines=True)

In [5]:
data_df.drop_duplicates(inplace=True)
data_df.dropna(inplace=True)
data_df.shape
data_df.drop('article_link',axis=1,inplace=True)
data_df.head(2)


,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0


In [6]:
X_train,X_test,Y_train,Y_test = train_test_split(data_df['headline'],data_df['is_sarcastic'],test_size=.7)
X_val,X_test,Y_Val,Y_test = train_test_split(X_test,Y_test,test_size=.5)

In [9]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
bert_model = AutoModel.from_pretrained("google-bert/bert-base-uncased")

In [10]:
class dataset(Dataset): 
    
    def __init__(self,X,Y):
        self.X =[tokenizer(x,max_length=100,truncation=True,padding='max_length',return_tensors='pt').to(device) for x in X]
        self.Y = torch.tensor(Y,dtype=torch.float32).to(device)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index] , self.Y[index]
    
    

In [11]:
train_dataset = dataset(X_train,Y_train.values)
val_dataset = dataset(X_val,Y_Val.values)
test_dataset = dataset(X_test,Y_test.values)

In [12]:
BATCH_SIZE =32
LR = 1e-4
EPOCHS = 10

In [13]:
train_dataloader = DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True)
val_dataloader = DataLoader(val_dataset,batch_size=BATCH_SIZE,shuffle=True)
test_dataloader = DataLoader(test_dataset,batch_size=BATCH_SIZE,shuffle=True)

In [14]:
class MyModel(nn.Module):
    def __init__(self,bert):
        super().__init__()
        self.bert = bert
        self.dropout = nn.Dropout(.25)
        self.linear1= nn.Linear(768,384)
        self.linear2 = nn.Linear(384,1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,input_ids,attention_mask):
        pooled_output = self.bert(input_ids,attention_mask,return_dict = False)[0][:,0]
        output = self.linear1(pooled_output)
        output=self.dropout(output)
        output = self.linear2(output)
        output = self.sigmoid(output)
        return output
        
        

In [15]:
for param in bert_model.parameters():
    param.requires_grad =False
mymodel = MyModel(bert_model).to(device)

In [16]:
criterion=  nn.BCELoss()
optimizer = Adam(mymodel.parameters(),lr=LR)


In [17]:
total_loss_train_plot = []
total_loss_val_plot = []
total_acc_train_plot = []
total_acc_val_plot = []

for epoch in range(EPOCHS):
    total_loss_train = 0
    total_loss_val=0
    total_acc_train = 0
    total_acc_val = 0
    
    for inputs,labels in train_dataloader:
        inputs.to(device)
        labels.to(device)
        
        mymodel.train()
        pred = mymodel(inputs['input_ids'].squeeze(1),inputs['attention_mask'].squeeze(1)).squeeze(1)
        batch_loss = criterion(pred,labels)
        total_loss_train+=batch_loss.item()
        
        acc=(pred.round()==labels).sum().item()
        
        total_acc_train +=acc
        batch_loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    with torch.no_grad():
        mymodel.eval()
        for inputs,labels in val_dataloader:
            inputs.to(device)
            labels.to(device)
            pred = mymodel(inputs['input_ids'].squeeze(1),inputs['attention_mask'].squeeze(1)).squeeze(1)
            batch_loss = criterion(pred,labels)
            total_loss_val += batch_loss.item()
            acc = (pred.round()==labels).sum().item()
            
            total_acc_val += acc
            
    total_loss_train_plot.append(round(total_loss_train/len(train_dataloader),4))
    total_loss_val_plot.append(round(total_loss_val/len(val_dataloader),4))
    total_acc_train_plot.append(round(total_acc_train/len(train_dataset)*100,4))
    total_acc_val_plot.append(round(total_acc_val/len(val_dataset)*100,4))
    
    print(f'Epoch:{epoch+1} Train Loss: {total_loss_train_plot[epoch]} Train Acc: {total_acc_train_plot[epoch]}%')
    print(f'Epoch:{epoch+1} Val Loss: {total_loss_val_plot[epoch]} Val Acc: {total_acc_val_plot[epoch]}%')
    
        

Epoch:1 Train Loss: 0.5298 Train Acc: 75.1997%
Epoch:1 Val Loss: 0.4465 Val Acc: 79.6213%
Epoch:2 Train Loss: 0.4424 Train Acc: 80.9661%
Epoch:2 Val Loss: 0.4046 Val Acc: 82.0496%
Epoch:3 Train Loss: 0.4156 Train Acc: 81.7649%
Epoch:3 Val Loss: 0.406 Val Acc: 81.5255%
Epoch:4 Train Loss: 0.405 Train Acc: 82.4014%
Epoch:4 Val Loss: 0.3713 Val Acc: 84.2747%
Epoch:5 Train Loss: 0.3957 Train Acc: 83.0005%
Epoch:5 Val Loss: 0.3655 Val Acc: 83.911%
Epoch:6 Train Loss: 0.3904 Train Acc: 83.0255%
Epoch:6 Val Loss: 0.3579 Val Acc: 84.7026%
Epoch:7 Train Loss: 0.3883 Train Acc: 83.0629%
Epoch:7 Val Loss: 0.3564 Val Acc: 84.8631%
Epoch:8 Train Loss: 0.3791 Train Acc: 83.9491%
Epoch:8 Val Loss: 0.3587 Val Acc: 84.157%
Epoch:9 Train Loss: 0.373 Train Acc: 83.9366%
Epoch:9 Val Loss: 0.3486 Val Acc: 85.0021%
Epoch:10 Train Loss: 0.3788 Train Acc: 83.7743%
Epoch:10 Val Loss: 0.352 Val Acc: 84.9914%


In [47]:
with torch.no_grad():
    mymodel.eval()
    total_loss_test = 0 
    total_acc_test = 0
    for inputs ,label in test_dataloader:
        inputs.to(device)
        label.to(device)
        prediction = mymodel(inputs['input_ids'].squeeze(1),inputs['attention_mask'].squeeze(1)).squeeze(1)
        batch_loss =criterion(prediction,label)
        total_loss_test +=batch_loss.item()
        acc = (prediction.round()==label).sum().item()
        
        total_acc_test +=acc
    print(f'Total acc for test set: {round(total_acc_test/len(test_dataset)*100,4)}%')
        

Total acc for test set: 85.0128%
